In [1]:
import pdfplumber
import pandas as pd

In [2]:

pdf_path = r"C:\Users\why19\Desktop\河北省环京津地区以水灭火等关键能力建设项目可行性研究报告（审定稿）(2).pdf"

In [3]:
# 函数：从表格上方提取表格名称
def extract_table_name(page, table_bbox):
    # 从表格上方提取文本区域
    top_area = (table_bbox[0], table_bbox[1] - 50, table_bbox[2], table_bbox[1])
    text = page.within_bbox(top_area).extract_text()
    return text.split('\n')[-1] if text else "Unknown Table Name"

In [ ]:
tables_dict = {}

with pdfplumber.open(pdf_path) as pdf:
    for page_number, page in enumerate(pdf.pages, start=1):
        for table_index, table in enumerate(page.extract_tables(), start=1):
            # 使用第一行作为表头创建 DataFrame
            df = pd.DataFrame(table[1:], columns=table[0])
            
            # 提取表格的边界框
            table_bbox = page.find_tables()[table_index - 1].bbox

            # 提取表格名称
            table_name = extract_table_name(page, table_bbox)

            # 添加表格名称列
            df["Table Name"] = table_name
            
            # 将 DataFrame 存储在字典中
            tables_dict[table_name] = df

In [ ]:
# 示例：显示提取的表格
for table_name, table_df in tables_dict.items():
    print(f"Table: {table_name}")
    print(table_df)
    print("\n")

# 将表格保存为单独的 CSV 文件，使用 `utf-8-sig` 编码以防止 Excel 打开时出现乱码
for table_name, table_df in tables_dict.items():
    csv_file = f"{table_name}.csv"
    table_df.to_csv(csv_file, index=False, encoding='utf-8-sig')